In [ ]:
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient  
from azure.search.documents.models import VectorizedQuery

import os
from dotenv import load_dotenv  
load_dotenv(override=True)

AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,  
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

index_name = os.getenv("AZURE_INDEX_NAME")
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
embedding_model = os.getenv('AZURE_OPENAI_EMBEDDING_MODEL')

credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY"))

def get_embeddings(text: str):
    embedding = client.embeddings.create(input=[text], model=embedding_model)
    return embedding.data[0].embedding

def single_vector_search():
    # [START single_vector_search]
    query = "What is Apple doing in AI space?"

    search_client = SearchClient(service_endpoint, index_name, credential)
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=1, fields="vector")

    results = search_client.search(
        vector_queries=[vector_query],
        select=["date", "content"],
    )

    for result in results:
        print(result)
    # [END single_vector_search]

def single_vector_search_with_filter():
    # [START single_vector_search_with_filter]
    query = "*"

    search_client = SearchClient(service_endpoint, index_name, credential)
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=3, fields="vector")

    results = search_client.search(
        search_text="",
        vector_queries=[vector_query],
        filter="date ge 2024-08-30T00:00:00Z",
        select=["date", "meta"],
    )

    for result in results:
        print(result)
    # [END single_vector_search_with_filter]


def simple_hybrid_search():
    # [START simple_hybrid_search]
    query = "What is the news about Sanford Health?"

    search_client = SearchClient(service_endpoint, index_name, credential)
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=1, fields="vector")

    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["date", "meta"],
    )

    for result in results:
        print(result)
    # [END simple_hybrid_search]

# Sample vector searches
single_vector_search()


In [ ]:
%pip install --upgrade --quiet langchain-community
%pip install --upgrade --quiet langchain-openai

In [29]:
from langchain_community.retrievers import AzureAISearchRetriever

import os
from dotenv import load_dotenv  
load_dotenv(override=True)

retriever: AzureAISearchRetriever = AzureAISearchRetriever(
    # api_key=api_key, service_name=service_name, index_name=index_name,
    content_key="content", top_k=3, filter="date ge 2024-08-30T00:00:00Z"
)

retriever.invoke("*")


[Document(metadata={'@search.score': 1.0, 'id': '443d307266234221822962a1d2180df6b74af928e41d82c89989b292e11cbf15', 'meta': '{"headline": "Sanford Health To End Participation With Humana Medicare Advantage On Dec. 31 Due To Coverage Denials And Care Delays; Patients Encouraged To Consider Other Plans During Medicare Open Enrollment", "author": "Benzinga Newsdesk", "symbols": "[\'HUM\']", "url": "https://www.benzinga.com/news/24/08/40660618/sanford-health-to-end-participation-with-humana-medicare-advantage-on-dec-31-due-to-coverage-denials", "model": "text-embedding-3-small", "usage_prompt_tokens": 36, "usage_total_tokens": 36}', 'date': '2024-08-30T19:25:38Z'}, page_content='Sanford Health To End Participation With Humana Medicare Advantage On Dec 31 Due To Coverage Denials And Care Delays Patients Encouraged To Consider Other Plans During Medicare Open Enrollment'),
 Document(metadata={'@search.score': 1.0, 'id': '1b04fb14a066628ba98087ed9f46e6c6e4a2028fb80a06a691affdde777d5b55', 'met

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from langchain_community.retrievers import AzureAISearchRetriever

import os
from dotenv import load_dotenv  
load_dotenv(override=True)

retriever: AzureAISearchRetriever = AzureAISearchRetriever(
    # api_key=api_key, service_name=service_name, index_name=index_name,
    content_key="content", top_k=1
)

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

    Context: {context}

    Question: {question}"""
)

llm = ChatOpenAI(model="gpt-4o-mini")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Who is investing in OpenAI?")

'Apple, Nvidia, and Microsoft are among the companies investing in OpenAI.'